In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

ROOT = Path("/Users/yizj/Desktop/HBN/pheno_assessment").resolve()
RAW_DIR = ROOT / "data" / "raw"
INTERIM_DIR = ROOT / "data" / "interim"
SYNTH_DIR = ROOT / "data" / "synth"
REPORTS_DIR = ROOT / "data" / "reports"

for d in [INTERIM_DIR, SYNTH_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

RAW_DIR, INTERIM_DIR, SYNTH_DIR, REPORTS_DIR


(PosixPath('/Users/yizj/Desktop/HBN/pheno_assessment/data/raw'),
 PosixPath('/Users/yizj/Desktop/HBN/pheno_assessment/data/interim'),
 PosixPath('/Users/yizj/Desktop/HBN/pheno_assessment/data/synth'),
 PosixPath('/Users/yizj/Desktop/HBN/pheno_assessment/data/reports'))

In [2]:
MISSING_TOKENS = {"", "NULL", "null", "NaN", "nan"}

def load_hbn_assessment(path: Path, standardize_cols=True) -> pd.DataFrame:
    # read all as strings first (stable)
    df = pd.read_csv(path, dtype=str, keep_default_na=False)

    if "Identifiers" not in df.columns:
        raise ValueError(f"{path.name}: expected column 'Identifiers' not found.")

    # drop template row(s): Identifiers like ",assessment"
    df = df[~df["Identifiers"].str.startswith(",", na=False)].copy()

    # split Identifiers -> subject_id,eventname
    parts = df["Identifiers"].str.split(",", n=1, expand=True)
    df["subject_id"] = parts[0].str.strip()
    df["eventname"]  = parts[1].str.strip()

    # normalize missing tokens
    df = df.replace({tok: np.nan for tok in MISSING_TOKENS})

    # standardize header commas for code safety (optional)
    if standardize_cols:
        df.columns = [c.replace(",", "_") for c in df.columns]

    return df

In [3]:
def numericize(df: pd.DataFrame, cols):
    out = df.copy()
    for c in cols:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

In [4]:
import re

META_HINTS = [
    "Administration", "Comment", "Data_entry", "Days_Baseline", "EID", "PSCID",
    "START_DATE", "Season", "Site", "Study", "Visit_label", "Year"
]

# Item patterns across instruments:
# CBCL: CBCL_01, CBCL_113A, CBCLpre_100a, etc.
# These are "item-level" (not aggregates).
ITEM_PATTERNS = [
    re.compile(r".*_[0-9]+[A-Za-z]*$"),          # ..._01, ..._113A
    re.compile(r".*_100[a-z]$"),                 # ..._100a, ..._100b
    re.compile(r".*_(pre_)?[0-9]+[A-Za-z]*$"),   # a bit more permissive for pre variants
]

def is_meta_col(col: str) -> bool:
    return any(h in col for h in META_HINTS) or col in {"Identifiers", "subject_id", "eventname"}

_item_index_re = re.compile(r"^(?P<prefix>.+)_(?P<idx>\d{1,3})(?P<suf>[A-Za-z]?)$")

def is_item_col(col: str, prefix: str) -> bool:
    if not col.startswith(prefix + "_"):
        return False
    m = _item_index_re.match(col)
    if not m:
        return False
    idx = int(m.group("idx"))
    # plausible item index range; adjust if needed
    return 1 <= idx <= 250

def get_aggregate_cols(df: pd.DataFrame, prefix: str):
    """
    Aggregate columns = instrument-derived summary scores/subscales/etc.
    Includes things like:
      - NLES_P_TotalEvents, NLES_P_Aware, NLES_P_Upset_Total, ...
      - SCARED_P_GD, SCARED_P_Total, ...
      - CBCL_Pre_DSM_ADHP, CBCL_Pre_Total_T, CBCL_Pre_OP, ...
    Excludes metadata and item-level responses.
    """
    cols = []
    for c in df.columns:
        if is_meta_col(c):
            continue
        if not c.startswith(prefix + "_"):
            continue
        if is_item_col(c, prefix):
            continue
        cols.append(c)
    return cols

def extract_aggregates(df: pd.DataFrame, file_tag: str) -> pd.DataFrame:
    prefix = infer_prefix_with_check(df)
    agg_cols = get_aggregate_cols(df, prefix)

    # numeric conversion only for aggregate cols
    df_num = numericize(df, agg_cols)

    out = df_num[["subject_id", "eventname"]].copy()

    for c in agg_cols:
        out[f"{file_tag}__{c}"] = df_num[c]

    return out

In [5]:
from collections import Counter

def infer_prefix(df: pd.DataFrame) -> str:
    admin_cols = [c for c in df.columns if c.endswith("_Administration")]
    if admin_cols:
        return admin_cols[0].replace("_Administration", "")

    # fallback: most frequent prefix among non-meta columns
    candidates = []
    for c in df.columns:
        if c in {"Identifiers", "subject_id", "eventname"}:
            continue
        if is_meta_col(c):
            continue
        if "_" in c:
            candidates.append("_".join(c.split("_")[:2]) if c.count("_") >= 2 else c.split("_")[0])
    if candidates:
        return Counter(candidates).most_common(1)[0][0]
    return "INSTR"

def infer_prefix_with_check(df):
    prefix = infer_prefix(df)
    agg = get_aggregate_cols(df, prefix)
    if len(agg) == 0:
        # fallback: try the most frequent 1-token prefix as well
        one_tok = []
        for c in df.columns:
            if is_meta_col(c): 
                continue
            if "_" in c:
                one_tok.append(c.split("_")[0])
        if one_tok:
            prefix2 = Counter(one_tok).most_common(1)[0][0]
            agg2 = get_aggregate_cols(df, prefix2)
            if len(agg2) > 0:
                return prefix2
    return prefix

def sanity_keys(df, name="df"):
    assert "subject_id" in df.columns and "eventname" in df.columns, f"{name}: missing keys"
    assert df["subject_id"].notna().all(), f"{name}: null subject_id"
    # eventname can be null if parsing fails; flag it:
    if df["eventname"].isna().mean() > 0:
        print(f"WARNING {name}: eventname missing rate = {df['eventname'].isna().mean():.3f}")
    dup = df.duplicated(subset=["subject_id","eventname"]).mean()
    if dup > 0:
        print(f"WARNING {name}: duplicate key rate = {dup:.4f}")

In [9]:
files = [
    "AdverseChildhoodExperiencesScale_child.csv",
    "AdverseChildhoodExperiencesScale_parent.csv",
    "ChildBehaviorChecklist_parent.csv",
    "ChildBehaviorChecklist_parentPreschool.csv",
    "ChildrensDepressionInventory_child.csv",
    "ChildrensDepressionInventory_parent.csv",
    "ConnersADHDRatingScales.csv",
    "ConnersAdultADHDRatingScales.csv",
    "NegativeLifeEventsScale_child.csv",
    "NegativeLifeEventsScale_parent.csv",
    "ScreenForChildAnxietyRelatedDisorders_child.csv",
    "ScreenForChildAnxietyRelatedDisorders_parent.csv",
    "StateTraitAnxietyInventory.csv",
]

dfs = {}
for fn in files:
    p = RAW_DIR / fn
    if not p.exists():
        print("MISSING:", fn)
        continue
    df = load_hbn_assessment(p, standardize_cols=True)
    dfs[fn] = df
    print(fn, df.shape, "prefix:", infer_prefix(df))

/var/folders/4_/vsf2fnhd3bz458z2nvlnyrmm0000gn/T/ipykernel_14967/54876077.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({tok: np.nan for tok in MISSING_TOKENS})
/var/folders/4_/vsf2fnhd3bz458z2nvlnyrmm0000gn/T/ipykernel_14967/54876077.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({tok: np.nan for tok in MISSING_TOKENS})
/var/folders/4_/vsf2fnhd3bz458z2nvlnyrmm0000gn/T/ipykernel_14967/54876077.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a

AdverseChildhoodExperiencesScale_child.csv (77, 26) prefix: ACE
AdverseChildhoodExperiencesScale_parent.csv (2197, 33) prefix: ACE_P
ChildBehaviorChecklist_parent.csv (4301, 161) prefix: CBCL
ChildBehaviorChecklist_parentPreschool.csv (161, 146) prefix: CBCL_Pre
ChildrensDepressionInventory_child.csv (175, 57) prefix: CDI_SR
ChildrensDepressionInventory_parent.csv (210, 92) prefix: CDI_P
ConnersADHDRatingScales.csv (3256, 66) prefix: C3SR
ConnersAdultADHDRatingScales.csv (121, 51) prefix: CAARS
NegativeLifeEventsScale_child.csv (1251, 85) prefix: NLES_SR
NegativeLifeEventsScale_parent.csv (4148, 82) prefix: NLES_P
ScreenForChildAnxietyRelatedDisorders_child.csv (3187, 62) prefix: SCARED_SR


/var/folders/4_/vsf2fnhd3bz458z2nvlnyrmm0000gn/T/ipykernel_14967/54876077.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({tok: np.nan for tok in MISSING_TOKENS})
/var/folders/4_/vsf2fnhd3bz458z2nvlnyrmm0000gn/T/ipykernel_14967/54876077.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({tok: np.nan for tok in MISSING_TOKENS})
/var/folders/4_/vsf2fnhd3bz458z2nvlnyrmm0000gn/T/ipykernel_14967/54876077.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a

ScreenForChildAnxietyRelatedDisorders_parent.csv (4177, 151) prefix: CDI_P
StateTraitAnxietyInventory.csv (73, 61) prefix: STAI


/var/folders/4_/vsf2fnhd3bz458z2nvlnyrmm0000gn/T/ipykernel_14967/54876077.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({tok: np.nan for tok in MISSING_TOKENS})


In [10]:
df = dfs["ChildBehaviorChecklist_parentPreschool.csv"]
prefix = infer_prefix_with_check(df)
agg_cols = get_aggregate_cols(df, prefix)
prefix, len(agg_cols), agg_cols[:30]

('CBCL_Pre',
 29,
 ['CBCL_Pre_CBCLPre_AB',
  'CBCL_Pre_CBCLPre_AB_T',
  'CBCL_Pre_CBCLPre_AD',
  'CBCL_Pre_CBCLPre_AD_T',
  'CBCL_Pre_CBCLPre_AP',
  'CBCL_Pre_CBCLPre_AP_T',
  'CBCL_Pre_CBCLPre_DSM_ADHP',
  'CBCL_Pre_CBCLPre_DSM_ADHP_T',
  'CBCL_Pre_CBCLPre_DSM_AP',
  'CBCL_Pre_CBCLPre_DSM_AP_T',
  'CBCL_Pre_CBCLPre_DSM_AnxP',
  'CBCL_Pre_CBCLPre_DSM_AnxP_T',
  'CBCL_Pre_CBCLPre_DSM_ODP',
  'CBCL_Pre_CBCLPre_DSM_ODP_T',
  'CBCL_Pre_CBCLPre_DSM_PDP',
  'CBCL_Pre_CBCLPre_DSM_PDP_T',
  'CBCL_Pre_CBCLPre_Ext',
  'CBCL_Pre_CBCLPre_Ext_T',
  'CBCL_Pre_CBCLPre_Int',
  'CBCL_Pre_CBCLPre_Int_T',
  'CBCL_Pre_CBCLPre_OP',
  'CBCL_Pre_CBCLPre_SC',
  'CBCL_Pre_CBCLPre_SC_T',
  'CBCL_Pre_CBCLPre_SP',
  'CBCL_Pre_CBCLPre_SP_T',
  'CBCL_Pre_CBCLPre_Total',
  'CBCL_Pre_CBCLPre_Total_T',
  'CBCL_Pre_CBCLPre_WD',
  'CBCL_Pre_CBCLPre_WD_T'])

 Extract per-instrument aggregate tables (in memory)

In [11]:
agg_by_file = {}
meta_rows = []

for fn, df in dfs.items():
    tag = Path(fn).stem  # e.g., ChildBehaviorChecklist_parent
    agg = extract_aggregates(df, file_tag=tag)

    # basic key sanity
    sanity_keys(agg, name=tag)

    # duplicate key rate
    dup_rate = agg.duplicated(subset=["subject_id", "eventname"]).mean()

    meta_rows.append({
        "file": fn,
        "tag": tag,
        "n_rows": len(agg),
        "n_cols": agg.shape[1],
        "n_agg_cols": agg.shape[1] - 2,
        "dup_key_rate": dup_rate,
        "eventname_unique": agg["eventname"].nunique(dropna=True),
    })

    agg_by_file[fn] = agg
    print(f"{tag}: {agg.shape} | dup_key_rate={dup_rate:.4f}")

meta_df = pd.DataFrame(meta_rows).sort_values("n_agg_cols", ascending=False)
meta_df.head(20)

AdverseChildhoodExperiencesScale_child: (77, 3) | dup_key_rate=0.0000
AdverseChildhoodExperiencesScale_parent: (2197, 3) | dup_key_rate=0.0000
ChildBehaviorChecklist_parent: (4301, 26) | dup_key_rate=0.0000
ChildBehaviorChecklist_parentPreschool: (161, 31) | dup_key_rate=0.0000
ChildrensDepressionInventory_child: (175, 16) | dup_key_rate=0.0000
ChildrensDepressionInventory_parent: (210, 8) | dup_key_rate=0.0000
ConnersADHDRatingScales: (3256, 14) | dup_key_rate=0.0000
ConnersAdultADHDRatingScales: (121, 12) | dup_key_rate=0.0000
NegativeLifeEventsScale_child: (1251, 6) | dup_key_rate=0.0000
NegativeLifeEventsScale_parent: (4148, 6) | dup_key_rate=0.0000
ScreenForChildAnxietyRelatedDisorders_child: (3187, 8) | dup_key_rate=0.0000
ScreenForChildAnxietyRelatedDisorders_parent: (4177, 8) | dup_key_rate=0.0000
StateTraitAnxietyInventory: (73, 8) | dup_key_rate=0.0000


,file,tag,n_rows,n_cols,n_agg_cols,dup_key_rate,eventname_unique
3,ChildBehaviorChecklist_parentPreschool.csv,ChildBehaviorChecklist_parentPreschool,161,31,29,0.0,1
2,ChildBehaviorChecklist_parent.csv,ChildBehaviorChecklist_parent,4301,26,24,0.0,1
4,ChildrensDepressionInventory_child.csv,ChildrensDepressionInventory_child,175,16,14,0.0,1
6,ConnersADHDRatingScales.csv,ConnersADHDRatingScales,3256,14,12,0.0,1
7,ConnersAdultADHDRatingScales.csv,ConnersAdultADHDRatingScales,121,12,10,0.0,1
5,ChildrensDepressionInventory_parent.csv,ChildrensDepressionInventory_parent,210,8,6,0.0,1
10,ScreenForChildAnxietyRelatedDisorders_child.csv,ScreenForChildAnxietyRelatedDisorders_child,3187,8,6,0.0,1
11,ScreenForChildAnxietyRelatedDisorders_parent.csv,ScreenForChildAnxietyRelatedDisorders_parent,4177,8,6,0.0,1
12,StateTraitAnxietyInventory.csv,StateTraitAnxietyInventory,73,8,6,0.0,1
8,NegativeLifeEventsScale_child.csv,NegativeLifeEventsScale_child,1251,6,4,0.0,1


Export REAL aggregate tables to data/interim/

Each file becomes: data/interim/<instrument>_aggregates.csv

In [12]:
written = []

for fn, agg in agg_by_file.items():
    tag = Path(fn).stem
    out_path = INTERIM_DIR / f"{tag}_aggregates.csv"
    agg.to_csv(out_path, index=False)
    written.append({"file": fn, "interim_path": str(out_path), "rows": len(agg), "cols": agg.shape[1]})

written_df = pd.DataFrame(written)
written_df.to_csv(REPORTS_DIR / "interim_aggregate_exports.csv", index=False)

print("Wrote", len(written_df), "aggregate tables to", INTERIM_DIR)
written_df.head(10)

Wrote 13 aggregate tables to /Users/yizj/Desktop/HBN/pheno_assessment/data/interim


,file,interim_path,rows,cols
0,AdverseChildhoodExperiencesScale_child.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,77,3
1,AdverseChildhoodExperiencesScale_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,2197,3
2,ChildBehaviorChecklist_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,4301,26
3,ChildBehaviorChecklist_parentPreschool.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,161,31
4,ChildrensDepressionInventory_child.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,175,16
5,ChildrensDepressionInventory_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,210,8
6,ConnersADHDRatingScales.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,3256,14
7,ConnersAdultADHDRatingScales.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,121,12
8,NegativeLifeEventsScale_child.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,1251,6
9,NegativeLifeEventsScale_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,4148,6


Synthetic per-instrument aggregates (same schema, safe IDs)

This preserves marginal distributions and missingness patterns. It also keeps joinability across instruments optional; since you’re doing per-instrument, we’ll generate IDs per instrument (privacy-safe). If later you want cross-instrument joins, tell me and I’ll switch to a shared synthetic ID spine.

In [13]:
def synth_bootstrap_table(df: pd.DataFrame, seed: int = 0, id_prefix="SYN"):
    rng = np.random.default_rng(seed)
    syn = df.sample(n=len(df), replace=True, random_state=seed).reset_index(drop=True)

    # replace subject_id with synthetic ids
    syn["subject_id"] = [f"{id_prefix}_{i:06d}" for i in range(len(syn))]

    return syn

synth_by_file = {}
written_synth = []

for i, (fn, real_agg) in enumerate(agg_by_file.items()):
    tag = Path(fn).stem
    syn = synth_bootstrap_table(real_agg, seed=42 + i, id_prefix=f"SYN_{tag[:10]}")

    # ensure schema match
    assert list(syn.columns) == list(real_agg.columns), f"Schema mismatch for {tag}"

    synth_by_file[fn] = syn

    out_path = SYNTH_DIR / f"{tag}_aggregates.csv"
    syn.to_csv(out_path, index=False)
    written_synth.append({"file": fn, "synth_path": str(out_path), "rows": len(syn), "cols": syn.shape[1]})

written_synth_df = pd.DataFrame(written_synth)
written_synth_df.to_csv(REPORTS_DIR / "synth_aggregate_exports.csv", index=False)

print("Wrote", len(written_synth_df), "synthetic aggregate tables to", SYNTH_DIR)
written_synth_df.head(10)

Wrote 13 synthetic aggregate tables to /Users/yizj/Desktop/HBN/pheno_assessment/data/synth


,file,synth_path,rows,cols
0,AdverseChildhoodExperiencesScale_child.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,77,3
1,AdverseChildhoodExperiencesScale_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,2197,3
2,ChildBehaviorChecklist_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,4301,26
3,ChildBehaviorChecklist_parentPreschool.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,161,31
4,ChildrensDepressionInventory_child.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,175,16
5,ChildrensDepressionInventory_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,210,8
6,ConnersADHDRatingScales.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,3256,14
7,ConnersAdultADHDRatingScales.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,121,12
8,NegativeLifeEventsScale_child.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,1251,6
9,NegativeLifeEventsScale_parent.csv,/Users/yizj/Desktop/HBN/pheno_assessment/data/...,4148,6


Validation report: missingness + summary stats
